In [1]:
import numpy as np
import cvxpy as cp
import itertools

In [2]:
class road_charge:
  def __init__(self, tau, detour, pcharging):
    self.tau = tau
    self.detour = detour
    self.pcharging = pcharging

In [3]:
class regulation_market:
  def __init__(self, Tr, Td, Tbar, price):
    self.Tr = Tr
    self.Td = Td
    self.Tbar = Tbar
    self.price = price

In [10]:
class EV_charge:
  def __init__(self, pmax, sfull, s0, sbar, tp):
    self.pmax = pmax
    self.sfull = sfull
    self.s0 = s0
    self.sbar = sbar
    self.tp = tp

  def charge_problem(self,road_charge):
    self.N = road_charge.detour.size
    self.b_charging = np.zeros(self.N)
    self.b_resting = np.zeros(self.N)
    self.ssafe = np.append(self.sbar*road_charge.detour.copy(),[0.])
    self.pactual = np.minimum(road.pcharging, self.pmax)

  def timid_policy(self):
    self.timid_charging = np.ones(self.N)
    self.timid_resting = np.ones(self.N)

  def greedy_policy(self,road_charge):
    b = np.zeros(self.N)
    s_current = self.s0
    tau = road_charge.tau
    detour = road_charge.detour
    ssafe = self.ssafe.copy()

    for i in range(self.N):
      if (s_current-self.sbar*(tau[i]+tau[i+1]))<self.ssafe[i+1]:
        b[i]=1
        s_current = self.sfull-detour[i]*self.sbar
      else:
        s_current -=self.sbar*tau[i]
  
    self.greedy_charging = b.copy()
    self.greedy_resting = b.copy()

  def base_policies(self,road_charge):
    self.charge_problem(road_charge)
    self.timid_policy()
    self.greedy_policy(road_charge)

  def time_schedule(self, b_charging, b_resting, road_charge, regulation_market):
    N = self.N
    tau = road_charge.tau.copy()
    detour = road_charge.detour.copy()

    pactual = self.pactual
    sfull = self.sfull
    ssafe = self.ssafe.copy()
    s0 = self.s0
    sbar = self.sbar
    tp = self.tp
    Tr = regulation_market.Tr
    Td = regulation_market.Td
    Tbar = regulation_market.Tbar
    Tbound = 1000
    price = regulation_market.price
    epsilon = 0.001

    b_auxiliary = np.logical_or(b_charging,b_resting).astype(float)
    #print(b_auxiliary)
    if detour.dot(b_auxiliary) <= (Tbar-tau.sum())/2:
      #print('hi')
      t_charging = cp.Variable((N))
      t_extra = cp.Variable((N))
      t_continuous_driving = cp.Variable((N+1))
      s_battery = cp.Variable((N+1))

      cost = 0
      constr = []
      constr += [t_continuous_driving[0] == tau[0]]
      constr += [s_battery[0] == s0-sbar*tau[0]]
      for i in range(N):
        constr += [t_continuous_driving[i]<= (Td-detour[i])]
        constr += [t_continuous_driving[i+1] == tau[i+1]+b_auxiliary[i]*detour[i]+(1-b_resting[i])*(t_continuous_driving[i]+b_auxiliary[i]*detour[i])]
        constr += [s_battery[i] >= ssafe[i]]
        constr += [t_charging[i] <= (sfull-(s_battery[i]-sbar*tau[i]))/pactual[i]]
        constr += [s_battery[i+1] == s_battery[i]+b_charging[i]*(t_charging[i]*pactual[i]-2*sbar*detour[i])-sbar*tau[i+1]]
        constr += [t_extra[i]+b_charging[i]*(t_charging[i]+tp) >= b_resting[i]*Tr]
        constr += [(1-b_resting[i])*b_charging[i]*(t_charging[i]+tp) <= Tr-epsilon]
        constr += [t_extra[i] >= 0, t_charging[i] >= 0]
        constr += [t_extra[i] <= b_resting[i]*Tbound, t_charging[i] <= b_charging[i]*Tbound]
        cost += t_extra[i]+b_charging[i]*(t_charging[i]+tp)+b_auxiliary[i]*2*detour[i]+b_charging[i]*t_charging[i]*price

      constr += [t_continuous_driving[N] <= Td, s_battery[N] >= ssafe[N]]
      #constr += [detour.dot(b_auxiliary) <= (Tbar-tau.sum())/2]

      problem = cp.Problem(cp.Minimize(cost), constr)
      op_value = problem.solve()
      op_t_charge = np.array(t_charging.value)
      op_t_extra = np.array(t_extra.value)
    else:
      op_value = np.inf
      op_t_charge = np.nan*np.ones(N)
      op_t_extra = np.nan*np.ones(N)
    return op_value, op_t_charge, op_t_extra

  def rollout(self, b_charging, b_resting, road_charge, regulation_market):
    rollout = np.inf
    temp = np.inf

    rollout_charging = b_charging.copy()
    rollout_resting = b_resting.copy()
    
    for i in range(self.N):
      b_charging = rollout_charging.copy()
      b_resting = rollout_resting.copy()
      for j in range(4):
        if j == 0:
          b_charging[i] = 0
          b_resting[i] = 0
        elif j == 1:
          b_charging[i] = 0
          b_resting[i] = 1
        elif j == 2:
          b_charging[i] = 1
          b_resting[i] = 0
        else:
          b_charging[i] = 1
          b_resting[i] = 1
        temp, op_t_charge, op_t_extra = self.time_schedule(b_charging, b_resting, road_charge, regulation_market)
        if rollout > temp:
          rollout_charging = b_charging.copy()
          rollout_resting = b_resting.copy()
          rollout = temp
    
    return rollout, rollout_charging, rollout_resting
    #self.rollout_charging =  rollout_charging.copy()
    #self.rollout_resting =  rollout_resting.copy()
    #self.rollout = rollout


  def best(self,road_charge, regulation_market):
    N = self.N
    best=np.inf
    current=np.inf
    best_charging=np.zeros(N)
    best_resting=np.zeros(N)

    vector=[0,1]
    b_charge_combinations=list(itertools.product(vector,repeat=N))# all the combinations of the binary variables for b_charging
    b_rest_combinations=list(itertools.product(vector,repeat=N))
    for b_1 in b_charge_combinations:
      for b_2 in b_rest_combinations:
        current,t_charge,t_extra=self.time_schedule(np.array(b_1), np.array(b_2),road_charge, regulation_market)
        if best > current:
          best_charging = np.array(b_1).copy()
          best_resting = np.array(b_2).copy()
          best_t_charge=t_charge.copy()
          best_t_extra=t_extra.copy()
          best = current.copy() 

    self.best_charging = best_charging.copy()
    self.best_resting = best_resting.copy()
    self.best_t_charge = best_t_charge.copy()
    self.best_t_extra = best_t_extra.copy()
    self.best = best


In [ ]:
a = np.append([1,2,3],[3])


In [11]:
# new testing data 
tau = np.array([19.9523, 96.6451, 78.43, 80.3998, 33.4749, 31.2283, 48.937])
detour = np.array([0.679, 14.9234, 6.1944, 10.2423, 3.6905, 0.5111])
#pcharging = np.array([175, 300, 300, 175, 175])/60
pcharging = np.array([300, 175, 300, 300, 175, 300])/60
pmax = 350/60
sfull = 468
s0 = 0.55*sfull
sbar = 110/60
#sbar = 107/60
ssafe = sbar*20
tp = 6
Tr = 45
Td = 4.5*60
Tbar = 9*60

price = 6

In [ ]:
s0-sbar*(tau[0]+tau[1]+tau[2])>sbar*detour[2]

True

In [ ]:
tau = np.array([32.15, 108.81, 24.35, 69.20, 65.64, 35.24])
detour = np.array([15.3, 10.1, 7.5, 8.3, 11.8])
#pcharging = np.array([175, 300, 300, 175, 175])/60
pcharging = np.array([300, 175, 300, 175, 175])/60
pmax = 350/60
sfull = 468
s0 = 200
sbar = 100/60
#sbar = 107/60
ssafe = sbar*20
tp = 6
Tr = 45
Td = 4.5*60
Tbar = 9*60

#price = 35
price = 35

In [ ]:
(32.15+108.81+24.35+7.5)/60

2.8801666666666668

In [ ]:
(108.81+24.35+69.20+65.64+15.3+11.8)/60

4.918333333333334

In [12]:
road = road_charge(tau, detour, pcharging)
regulation = regulation_market(Tr, Td, Tbar, price)
ev = EV_charge(pmax, sfull, s0, sbar, tp)

In [13]:
ev.base_policies(road)
print(ev.ssafe,ev.greedy_charging)
ev.time_schedule(ev.greedy_charging,ev.greedy_resting,road,regulation)

[ 1.24483333 27.35956667 11.3564     18.77755     6.76591667  0.93701667
  0.        ] [0. 1. 0. 0. 0. 1.]


(1203.0433400909885,
 array([-2.22247976e-11,  1.44626931e+02, -2.38365153e-11, -1.43440228e-11,
        -7.79792847e-12,  1.81309700e+01]),
 array([2.10389403e-09, 1.13872518e-08, 2.10389403e-09, 2.10389403e-09,
        2.10389403e-09, 2.08690300e+01]))

In [ ]:
ev.greedy_charging

array([0., 1., 0., 0., 0., 1.])

In [14]:
ev.rollout(ev.timid_charging,ev.timid_resting, road, regulation)

(699.2765800137637,
 array([1., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]))

In [15]:
ev.rollout(ev.greedy_charging,ev.greedy_resting, road, regulation)

(881.2450433524598,
 array([1., 1., 1., 0., 0., 1.]),
 array([1., 1., 0., 0., 0., 1.]))

In [ ]:
ev.time_schedule(ev.greedy_charging, ev.greedy_resting, road, regulation)

(inf, array(None, dtype=object), array(None, dtype=object))

In [ ]:
ev.best(road, regulation)


In [ ]:
print(ev.best,ev.best_charging,ev.best_resting)

699.2765800151599 [1 0 1 0 0 0] [0 0 1 0 0 0]
